In [ ]:
!pip3 install -U owlready2 rdflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 30.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 488.1 kB/s eta 0:00:00
  Created wheel for owlready2: filename=Owlready2-0.41-cp310-cp310-linux_x86_64.whl size=24170139 sha256=16e4cd54b69c5765071125b32f6df3433eb4f740645af4b6f89b2c57eab1aaa3
  Stored in directory: /root/.cache/pip/wheels/da/02/c0/a65e08daab6da8123998823c9aee7bba656b1500ab05d183ef
Successfully built owlready2


In [ ]:
import rdflib
from owlready2 import get_ontology
from rdflib import Graph
from rdflib.namespace import RDF, FOAF
from rdflib import Namespace
from rdflib import Literal, XSD

In [ ]:
#Information in the web of roadsings
graph  = Graph()
graph.parse("/content/rgb_roadsign_data.ttl")

#Ontology to populate (Our ontology)
Graph_onto = Graph()
onto = get_ontology("/content/DefinitivaSinPopular.rdf").load() #Our ontology. 

## Categorize colors

In [ ]:
from scipy.spatial import Voronoi, voronoi_plot_2d
import numpy as np
from ast import literal_eval

def categorize_colors(color_space, colors):
    """
    Categorize a list of colors in a color space according to Voronoi tessellation.
    :param color_space: a dictionary of prototype colors and their coordinates in the color space
    :param colors: a list of colors to categorize in the color space
    :return: a dictionary of the closest prototype color for each input color
    """
    # Create a list of coordinates for the prototype colors
    points = [v for k, v in color_space.items()]
    # Compute the Voronoi tessellation for the prototype colors
    vor = Voronoi(points)
    # Create a dictionary to store the closest prototype color for each input color
    categories = {}
    # Find the closest prototype color for each input color
    for color in colors:
        # Parse the color to a tuple if it's a string
        c = literal_eval(color) if isinstance(color, str) else color
        # Compute the distances between the input color and the prototype colors
        distances = [(i, np.linalg.norm(np.array(c) - np.array(p))) for i, p in enumerate(points)]
        # Sort the distances by distance value
        distances.sort(key=lambda x: x[1])
        # The closest prototype color is the one with the smallest distance
        closest = distances[0][0]
        # Add the closest prototype color to the categories dictionary
        categories[color] = list(color_space.keys())[closest]
    return categories



In [ ]:
color_space = {
    'RED': (255, 0, 0),
    'BLUE': (0, 0, 255),
    'BLACK': (0, 0, 0),
    'WHITE': (255, 255, 255)
}

## Just some testing:

In [ ]:
colors = ["(25,13,1)", "(100,150,200)"]

In [ ]:
categorize_colors(color_space, colors)

{'(25,13,1)': 'BLACK', '(100,150,200)': 'BLUE'}

In [ ]:
colors = [(255,255,0)]

In [ ]:
categorize_colors(color_space, colors)

{(255, 255, 0): 'RED'}

In [ ]:
list(categorize_colors(color_space, ["(0,23,250)"]).items())[0][1]

'BLUE'

##Ontology Population

In [ ]:
RSS = Namespace("http://www.iiia.csic.es/~marco/kr/roadsign-schema#")
instances = {}
for s, p, o in graph:   #graph is a list of tuples, each tuple is called a Triplet of the form (subject, predicate, object)
 if p == RDF.type and o == RSS.road_sign:  #If the predicate of a triplet is of type RDF and the object is a RoadSign
     instances[s] = onto.RoadSign()

In [ ]:
for s, p, o in graph:
  if p == FOAF.depicts:
    instances[o].filename = [s.toPython()]

  if s in instances:
    value = o.toPython().upper().replace(" ","_")
    value = value.replace("%", "percent")

    if value.startswith("("):
      value = value #I dont want the RGB tuples to be changed from , separated values to dot separated values
    else:
      value = value.replace(",", ".")
    
    if (p == RSS.shape):
      instances[s].shape = onto.Shape(value)

    if (p == RSS.ground_colour):
      if value.startswith("("):
        value = list(categorize_colors(color_space, [value]).items())[0][1]
        instances[s].groundColor = onto.Color(value)
      else:
        instances[s].groundColor = onto.Color(value)

    if (p == RSS.border_colour):
      if value.startswith("("):
        value = list(categorize_colors(color_space, [value]).items())[0][1]
        instances[s].borderColor =  [onto.Color(value)]
      else:
        instances[s].borderColor =  [onto.Color(value)]

    if (p == RSS.symbol):
      instances[s].symbol = [onto.Symbol(value)]

    if (p == RSS.symbol_colour):
      if value.startswith("("):
        value = list(categorize_colors(color_space, [value]).items())[0][1]
        instances[s].symbolColor =  [onto.Color(value)]
      else:
        instances[s].symbolColor = [onto.Color(value)]

    if (p == RSS.symbol_value):      
      instances[s].symbolValue = [value]


In [ ]:
onto.save("/content/DefinitivaPopulada")